In [ ]:
import os
os.chdir('..')
from notebooks import config
import geopandas as gpd
import numpy as np
from utils.geotif_io import readTiff, writeTiff
from utils.imgShow import imgShow
import matplotlib.pyplot as plt
from utils.transform_xy import coor2coor, imagexy2geo, geo2imagexy


In [ ]:
path_shp = 'data/tibet_tiles/tiles_shp/tibet_tiles.gpkg'
path_img_1 = 'data/tibet_tiles/s1_ascend/tibet_s1as_202008_tile_1.tif'
path_img_2 = 'data/tibet_tiles/s1_ascend/tibet_s1as_202008_tile_2.tif'
path_img_mosaic = 'data/tibet_tiles/mosaic_img.tif'


In [ ]:
!gdal_merge.py -o data/tibet_tiles/mosaic_img.tif -init 0 data/tibet_tiles/s1_ascend/tibet_s1as_202008_tile_1.tif data/tibet_tiles/s1_ascend/tibet_s1as_202008_tile_2.tif



In [ ]:
tiles_gdf = gpd.read_file(path_shp)
# tiles_gdf['geometry'][0]
tiles_gdf.loc[2:4]


In [ ]:
s1_ascend_t1, s1_ascend_info_t1 = readTiff(path_in=path_img_1)
s1_ascend_t2, s1_ascend_info_t2 = readTiff(path_in=path_img_2)
s1_ascend_mosaic, s1_ascend_info_mosaic = readTiff(path_in=path_img_mosaic)
print(s1_ascend_info_t1)
print(s1_ascend_info_t2)
print(s1_ascend_info_mosaic)


In [ ]:
plt.figure(figsize=(10,5))
imgShow(s1_ascend_mosaic, color_bands=(0,1,0), clip_percent=2)



In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
imgShow(s1_ascend_t1, color_bands=(0,1,0), clip_percent=2)
plt.subplot(1,2,2)
imgShow(s1_ascend_t2, color_bands=(0,1,0), clip_percent=2)


In [ ]:
extent_1 = s1_ascend_info_t1['geoextent']
extent_2 = s1_ascend_info_t2['geoextent']
print(extent_1)
print(extent_2)



In [ ]:
## common region
extents = np.array([extent_1, extent_2])
extents_min = np.min(extents, axis=0)
extents_max = np.max(extents, axis=0)
common_extent = [extents_max[0], extents_min[1], extents_max[2], extents_min[3]]
print(common_extent)
mosaic_extent = [extents_min[0], extents_max[1], extents_min[2], extents_max[3]]
print(mosaic_extent)


In [ ]:
pixel_width_1 = s1_ascend_info_t1['geotrans'][1]
pixel_height_1 = abs(s1_ascend_info_t1['geotrans'][5])
print(pixel_width_1, pixel_height_1)


In [ ]:
mosaic_cols = int((mosaic_extent[1]-mosaic_extent[0]) / pixel_width_1)
mosaic_rows = int((mosaic_extent[3]-mosaic_extent[2]) / pixel_height_1)
print(mosaic_rows, mosaic_cols)


In [ ]:
col_offset_1 = int((extent_1[0]-mosaic_extent[0])/pixel_width_1)
row_offset_1 = int((extent_1[3]-mosaic_extent[3])/pixel_height_1)
print(row_offset_1, col_offset_1)
col_offset_2 = int((extent_2[0]-mosaic_extent[0])/pixel_width_1)
row_offset_2 = int((extent_2[3]-mosaic_extent[3])/pixel_height_1)
print(row_offset_2, col_offset_2)

mosaic_geotrans = [mosaic_extent[0], pixel_width_1, 0, mosaic_extent[3], pixel_height_1, 0]
